In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [1]:
# import packages
import pandas as pd
df = pd.read_csv(r'/kaggle/input/netflix-shows/netflix_titles.csv')

import matplotlib.pyplot as plt

import seaborn as sns
sns.set(palette='deep')

import numpy as np

from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
import warnings
warnings.filterwarnings('ignore')

# A Brief Look Into Seasonality

Objectives:

-Inspect the dataset

-Cleanse the dataset as required

-Identify any trends between Season and when content is added to the platform

# Inspecting the data

In [11]:
#Checking the head
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [12]:
#Checking the column names, they are logical, no need to rename
df.columns


Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [13]:
#Checking how many rows and columns
df.shape

(8807, 12)

In [14]:
#making a copy before changes are made
df1=df.copy()

# Cleansing the data

In [15]:
#converting the date_added column to datetime type for easier analysis
df1['date_added'] = pd.to_datetime(df1['date_added'].str.strip(), format='%B %d, %Y')

In [16]:
df1['added_year'] = pd.to_datetime(df['date_added']).dt.year

ValueError: time data " August 4, 2017" doesn't match format "%B %d, %Y", at position 1442. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
#Creating a numerical column for month added
df1['month_added'] = pd.to_datetime(df1['date_added']).dt.month

In [ ]:
#Creating a numerical column for year added
df1['release_year'] = pd.to_datetime(df1['release_year'].astype(str), format='%Y')

In [ ]:
#Creating a numerical column 'turnaround', showing different between year released and year launched on Netflix
df1['turnaround'] = df1['date_added'] - df1['release_year']

In [ ]:
#Checking the changes have been made
df1.head()

# Seasonality
Identifying the Top 5 countries and creating a new df called msk_top5 to work with, they are all in the northern hemisphere so this will impact the function 

In [ ]:
#lots of the country entries are combined lists of countries, that's why there are so many different rows
df1['country'].value_counts().sum()

In [ ]:
# make a mask for france in country
df1["fr"]= df1["country"].str.contains("France", na=False)
is_fr = df1["fr"]== True
msk_fr= df1[is_fr]

# make a mask for canada in country
df1["can"]= df1["country"].str.contains("Canada", na=False)
is_can = df1["can"]== True
msk_can= df1[is_can]

#make a mask for india in country
df1["ind"]= df1["country"].str.contains("India", na=False)
is_ind = df1["ind"]== True
msk_ind= df1[is_ind]

#make a mask for united states in country
df1["US"]= df1["country"].str.contains("United States", na=False)
is_us = df1["US"]== True
msk_us= df1[is_us]

#make a mask for uk in country
df1["UK"]= df1["country"].str.contains("United Kingdom", na=False)
is_uk = df1["UK"]== True
msk_uk= df1[is_uk]

In [ ]:
#make a new df of the top 5 countries
msk_top5= pd.concat([msk_uk, msk_can, msk_us, msk_fr, msk_ind])
msk_top5.shape

# Adding a column for Season
To do this I made a function using an if loop to assign each month number to a season.

I then applied the function to the month_nummber column, creating a new column for season

In [ ]:
def find_season(x):
    
    if x == 12 or x==1 or x==2:
        return 'Winter'
    elif x == 3 or  x==4 or x==5:
        return 'Spring'
    elif x == 6 or x==7 or x==8:
        return 'Summer'
    elif x == 9 or x==10 or x==11:
        return 'Autumn'

In [ ]:
msk_top5['season'] =msk_top5['month_added'].apply(find_season)

In [ ]:
msk_top5['season'].value_counts()

In [ ]:
#Masking the data by season
msk_winter= msk_top5[msk_top5['season']== 'Winter']
msk_autumn= msk_top5[msk_top5['season']== 'Autumn']
msk_summer= msk_top5[msk_top5['season']== 'Summer']
msk_spring= msk_top5[msk_top5['season']== 'Spring']

# Visualisations

In [ ]:
plt.figure()
sns_season_fig =sns.countplot(data=msk_top5, x='season')
sns_season_fig.set(xlabel ="Season", ylabel = "Number of shows", title ='Number of Netflix Shows Launched by Season')

The barplots above show that Netflix consistantly release content over the year, with a 10% increase in Autumn and Winter. This supplies the demand, as people tend to watch more TV in Autumn and Winter.

In [ ]:
plt.figure()
type_fig = sns.countplot(data=df1, x='type')
type_fig.set(xlabel ="Type", ylabel = "Count", title ='Types of Shows Available on Netflix')

-Netflix shows can be divided into 2 types: TV show and Movie

-Overall, there are 39% more movies than TV shows. 

In [ ]:
#in the top 5 countries
plt.figure()
top5_type_fig = sns.countplot(data=msk_top5, x='type')
top5_type_fig.set(xlabel ="Type", ylabel = "Count", title ='Types of Shows Available in the Top 5 Countries')

The above chart shows that the ratio identified across the whole df is also applicable to the Top 5

In [ ]:
season_type_fig =  sns.FacetGrid(data= msk_top5, col = "season",
                    height = 4, aspect  =.5)
season_type_fig.map(sns.countplot, "type", order = ['Movie', 'TV Show'])
season_type_fig.fig.suptitle('Show Types Launched by Season', y=1.03)
season_type_fig.set(xlabel='Type', ylabel='Count')

Here we can see the composition of the content launched side by side for each season. 

Surprisingly the ratio of TV shows to Movies launched stays consistent across the seasons!

The composition only differs by a maximum of 2% per season.


# Conclusion

-Netflix launch shows onto their platform consistently across the year. This allows them to stay fresh, relevant and talked about.
In Autumn and Winter months, they increase the number of shows launched by 10%, this supplies the demand as people tend to watch more tv at this time.
This pattern allows the platform to be fresh, consistent and supply the demand according to season.

-Netflix shows 40% more Movies than TV Shows at all times in a consistent ratio. This maintains structural consistency of the brand and increases customer loyalty to the brand.
